In [1]:
from Estimator import NeuralPRFEstimator
import pyterrier as pt
import os
os.environ["JAVA_HOME"] = "/local/trmaster/opt/jdk-11.0.6/"
if not pt.started():
    pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"], mem=8000)
pt.cache.CACHE_DIR = "/users/tr.%s/pyterrier_transformer_cache" % (os.environ["USER"])

Using TensorFlow backend.


In [2]:
topics = pt.get_dataset("trec-robust-2004").get_topics()

In [3]:
topicsTrain=topics[topics.qid.astype(int) % 2 ==0]
topicsTest=topics[topics.qid.astype(int) % 2 !=0]

In [4]:
qrels = pt.get_dataset("trec-robust-2004").get_qrels()

In [5]:
pt.ApplicationSetup.setProperty("termpipelines", "")
index_location = "/users/tr.craigm/projects/tr5/Terrier/var/index_disk45/data.properties"

eval_metrics = ["map", "num_rel_ret", "ndcg", "ndcg_cut_10", "recip_rank"]

In [6]:
index = pt.IndexFactory.of(index_location)
print(index.getCollectionStatistics().toString())

Number of documents: 528155
Number of terms: 520423
Number of fields: 0
Number of tokens: 143669447
Field names: []
Positions:   false



In [7]:
from pyterrier.rewrite import RM3, AxiomaticQE

firstpass_dph = pt.BatchRetrieve(index, wmodel="DPH", 
                                 properties={"termpipelines": ""},
                                 verbose=True, metadata=["docno"])

In [8]:
res_train=(firstpass_dph % 100).transform(topicsTrain)
res_test= (firstpass_dph % 100).transform(topicsTest)

100%|██████████| 125/125 [00:03<00:00, 34.00q/s]


In [9]:
res_train.shape, res_test.shape

((9803, 6), (8690, 6))

In [10]:
import os
nrf = NeuralPRFEstimator(index,'data_robust_100')

In [11]:
# nrf.preprocess(res_train, qrels, topicsTrain, 20)

In [12]:
relevance_dict=nrf.create_relevance(res_train,qrels)
topk_term_corpus = nrf.top_k(res_train,20)
top_idf_dict = nrf.parse_idf_for_document(relevance_dict,topk_term_corpus, rerank_topk=60, doc_topk_term=20)


In [ ]:
nrf.fit(res_train,qrels,topicsTrain)

2020-08-03 02:00:30,519 : WARNING : From /users/tr.saraw/anaconda/envs/pyterrier/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



2020-08-03 02:00:30,526 : WARNING : From /users/tr.saraw/anaconda/envs/pyterrier/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



2020-08-03 02:00:30,591 : WARNING : From /users/tr.saraw/anaconda/envs/pyterrier/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



2020-08-03 02:00:30,610 : WARNING : From /users/tr.saraw/pytrnprf_100_file/model/nprf_drmm.py:66: The name tf.matrix_diag_part is deprecated. Please use tf.linalg.diag_part instead.

2020-08-03 02:00:30,641 : INFO : Apply doc gating, No dense layer on top, sum up scores


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dd_d_input (InputLayer)         (None, 20, 20, 10)   0                                            
__________________________________________________________________________________________________
dd_q_input (InputLayer)         (None, 20, 20, 1)    0                                            
__________________________________________________________________________________________________
hidden (Dense)                  (None, 20, 20, 5)    55          dd_d_input[0][0]                 
__________________________________________________________________________________________________
dd_q_gate (Dense)               (None, 20, 20, 1)    1           dd_q_input[0][0]                 
__________________________________________________________________________________________________
dd_d_gate 

...

[Valid]		MAP	P20	NDCG20
		0.0798	0.1725	0.2023
Epoch 90
iteration: 539
[Iter 540]	Loss: [0.5673744358681878]
[Valid]		MAP	P20	NDCG20
		0.0779	0.1625	0.1958
iteration: 540
[Iter 541]	Loss: [0.567171632212848]
[Valid]		MAP	P20	NDCG20
		0.0776	0.1600	0.1950
iteration: 541
[Iter 542]	Loss: [0.5672339054317813]
[Valid]		MAP	P20	NDCG20
		0.0774	0.1600	0.1947
iteration: 542
[Iter 543]	Loss: [0.5667869739322794]
[Valid]		MAP	P20	NDCG20
		0.0769	0.1575	0.1929
iteration: 543
[Iter 544]	Loss: [0.5669281141088885]
[Valid]		MAP	P20	NDCG20
		0.0773	0.1575	0.1947
iteration: 544
[Iter 545]	Loss: [0.5670654429159617]


2020-08-03 04:33:46,574 : INFO : Generate totally 10120 pairs


[Valid]		MAP	P20	NDCG20
		0.0773	0.1600	0.1964
Epoch 91
iteration: 545
[Iter 546]	Loss: [0.5636768739655791]
[Valid]		MAP	P20	NDCG20
		0.0767	0.1525	0.1889
iteration: 546
[Iter 547]	Loss: [0.5634163262050143]
[Valid]		MAP	P20	NDCG20
		0.0766	0.1550	0.1904
iteration: 547
[Iter 548]	Loss: [0.5633198565880772]
[Valid]		MAP	P20	NDCG20
		0.0768	0.1600	0.1953
iteration: 548
[Iter 549]	Loss: [0.5633311756102464]
[Valid]		MAP	P20	NDCG20
		0.0768	0.1575	0.1928
iteration: 549
[Iter 550]	Loss: [0.5633499714814627]
[Valid]		MAP	P20	NDCG20
		0.0767	0.1575	0.1925
iteration: 550
[Iter 551]	Loss: [0.5632684328280418]


2020-08-03 04:35:29,282 : INFO : Generate totally 10120 pairs


[Valid]		MAP	P20	NDCG20
		0.0767	0.1575	0.1925
Epoch 92
iteration: 551
[Iter 552]	Loss: [0.5659937491058832]
[Valid]		MAP	P20	NDCG20
		0.0787	0.1700	0.1995
iteration: 552


In [15]:
print('ok')

ok


In [ ]:
nrf.preprocess(res_test, qrels, topicsTest, 20)

2020-08-04 08:00:12,871 : INFO : loading projection weights from data/GoogleNews-vectors-negative300.bin
2020-08-04 08:01:27,781 : INFO : loaded (3000000, 300) matrix from data/GoogleNews-vectors-negative300.bin
 31%|███▏      | 30/96 [1:59:15<5:34:42, 304.28s/it]